<a href="https://colab.research.google.com/github/Hitomiblood/ProjectsDataScienceMaster/blob/main/Proyecto_TFM/Models/MotileDetection/PostBestModel/TrackingBytetrack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generando Video de Tracking con modelo Small

In [1]:
%%capture
!pip install ultralytics

In [2]:
from ultralytics import YOLO

In [3]:
%%capture
!wget https://www.dropbox.com/scl/fi/maqbuw2e0s7ywa63dofun/bestSmall1920.pt?rlkey=n0pqsnjgycak6v7jx3qzhladt&st=04t3nonx&dl=0
!mv bestSmall1920.pt?rlkey=n0pqsnjgycak6v7jx3qzhladt bestSmall1920.pt

In [4]:
# Load a pretrained YOLO model (recommended for training)
model = YOLO('bestSmall1920.pt')

In [5]:
%%capture
# Descargar el archivo ZIP desde la URL
!wget https://www.dropbox.com/scl/fi/vnyl0lbmzarkfm7jrt88h/Videos.zip?rlkey=4i3y4so9xnomins88tpgdls0z&st=mgjlmjb3&dl=0

# Crear la carpeta "Videos" si no existe
!mkdir -p Videos

!mv Videos.zip?rlkey=4i3y4so9xnomins88tpgdls0z Videos.zip

# Descomprimir el archivo ZIP en la carpeta "Videos"
!unzip -o Videos.zip -d Videos

In [6]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from collections import defaultdict
import os

cv2.imshow = cv2_imshow


In [35]:
# Directorio de entrada y salida
input_folder = "Videos/"
output_folder = "output_videos/"

# Crear el directorio de salida si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Listar los archivos de video en el directorio de entrada
video_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

# Iterar sobre cada archivo de video
for video_file in video_files:
    # Abrir el archivo de video de entrada
    video_path = os.path.join(input_folder, video_file)
    cap = cv2.VideoCapture(video_path)

    # Verificar si la apertura del video fue exitosa
    if not cap.isOpened():
        print(f"Error al abrir el archivo de video: {video_file}")
        continue

    # Obtener información del video de entrada
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_size = (frame_width, frame_height)

    # Crear el objeto VideoWriter para el video de salida
    output_video_file = os.path.join(output_folder, f"{os.path.splitext(video_file)[0]}_Out.avi")
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # Formato de compresión de vídeo
    out = cv2.VideoWriter(output_video_file, fourcc, fps, frame_size)

    # Store the track history
    track_history = defaultdict(lambda: [])

    model = YOLO('bestSmall1920.pt')

    # Loop through the video frames
    while cap.isOpened():
        # Leer un cuadro del video
        success, frame = cap.read()

        if success:
            # Ejecutar el seguimiento YOLOv8 en el cuadro, persistiendo los tracks entre cuadros
            results = model.track(frame, persist=True, tracker="bytetrack.yaml")

            # Verificar si se detectaron resultados
            if results and len(results) > 0 and len(results[0].boxes) > 0:
                # Obtener las cajas y los IDs de seguimiento
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()

                # Visualizar los resultados en el cuadro
                annotated_frame = results[0].plot()

                # Dibujar las líneas de seguimiento
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]
                    track.append((float(x), float(y)))  # x, y centro
                    if len(track) > 80:  # mantiene 80 tracks por 80 cuadros
                        track.pop(0)

                    # Dibujar las líneas de seguimiento
                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 255, 255), thickness=3)

                # Escribir el cuadro procesado en el archivo de salida
                out.write(annotated_frame)
            else:
                # Si no se detectaron resultados, escribir el cuadro original en el archivo de salida
                out.write(frame)

        else:
            # Terminar la ejecución si se alcanza el final del video
            break

    # Liberar los recursos
    cap.release()
    out.release()


requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.9 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 8.4s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 1216x1920 16 Movils, 3737.7ms
Speed: 103.9ms preprocess, 3737.7ms inference, 1702.8ms postprocess per image at shape (1, 3, 1216, 1920)

0: 1216x1920 16 Movils, 4457.7ms
Speed: 14.4ms preprocess, 4457.7ms inference, 2.1ms postprocess per image at shape (1, 3, 1216, 1920)

0: 1216x1920 16 Movils, 3243.8ms
Speed: 18.5ms preprocess, 3243.8ms inference, 1.7ms postprocess per image at shape (1, 3, 1216, 1920)

0: 1216x1920 16 Movils, 3167.1ms
Speed: 15.1ms preprocess, 3167.1ms inference, 1.6ms postprocess per image at shape (1, 3, 1216, 1920)

0: 1216x1920 16 Movils, 3274.2ms
Speed: 13.5ms preprocess, 3274.2ms inference, 1.7ms postprocess per image at shape (1, 3, 12

In [36]:
import shutil
archivo = shutil.make_archive('VideosOutTrackingSmall', 'zip','/content/output_videos')